In [60]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timezone
from selenium import webdriver
import csv
import json

In [61]:
DEFAULT_APACHE_PROJECT_URL = 'https://issues.apache.org/jira/browse/CAMEL'
SPECIFIC_ISSUE_NO = None # Do not modify this . Modify start and end
ISSUE_NO_START = 1
ISSUE_NO_END = 10597


if ISSUE_NO_START == ISSUE_NO_END:
   SPECIFIC_ISSUE_NO = ISSUE_NO_END

In [62]:
def time_epoch(date):
    dt = datetime.strptime(date, "%d/%b/%y %H:%M").replace(tzinfo=timezone.utc)
    epoch = int(dt.timestamp())
    return epoch

In [63]:
def fetch_issue_body(issue_url):
    res = requests.get(issue_url)
    soup = BeautifulSoup(res.text, 'html.parser')

    type_tag = soup.find('span', {'id': 'type-val', 'class': 'value'})
    type = type_tag.text.strip() if type_tag else None

    assignee_tag = soup.find('span', {'class':'user-hover'})
    assignee = assignee_tag.text.strip() if assignee_tag else None

    created_tag = soup.find('span', {'id': 'created-val', 'data-fieldtype': 'datetime'})
    created_at = created_tag.text.strip() if created_tag else None

    created_at_epoch = time_epoch(created_at) if created_at else None

    desc_tag = soup.find('div', {'id':'description-val'})
    description = desc_tag.get_text(strip=True) if desc_tag else None

    return {
        'type' : type,
        'assignee' : assignee,
        'createdAt' : created_at,
        'created_at_epoch' : created_at_epoch,
        'description' : description
    }

In [64]:
def fetch_issue_comments(issue_url):
    driver = webdriver.Chrome()
    driver.get(issue_url)
    html = driver.page_source
    driver.quit()
    soup = BeautifulSoup(html, 'html.parser')

    comments_sec = soup.find_all('div', {'class':'issue-data-block activity-comment twixi-block expanded'})
    comments = []

    for sec in comments_sec:
        comment_sec = sec.find('div', {'class':'twixi-wrap verbose actionContainer'})
        
        author_tag = comment_sec.find('a', {'class' : 'user-hover user-avatar'})
        author = author_tag.text.strip() if author_tag else None

        time_tag = comment_sec.find('time')
        created_at = time_tag.text.strip() if time_tag else None
        created_at_epoch = time_epoch(created_at) if created_at else None

        body_tag = comment_sec.find('div', {'class':'action-body flooded'})
        text = body_tag.get_text(strip=True) if body_tag else None

        comment = {
            "author" : author,
            "created_at":created_at,
            "created_at_epoch":created_at_epoch,
            "text": text
        }
        comments.append(comment)
    return comments

In [65]:
issue_no = ISSUE_NO_START
issues = []

while issue_no <= ISSUE_NO_END:
    issue_url = f'{DEFAULT_APACHE_PROJECT_URL}-{issue_no}'
    try:
        issue = fetch_issue_body(issue_url)
        comments = fetch_issue_comments(issue_url)
        issue['comments'] = comments
        issue['id'] = issue_no
        issue['url'] = issue_url
        issues.append(issue)
        print(f"issue done: {issue_no}")
    except Exception as e:
        print(f"Error in issue {issue_no}: {e}")
    issue_no +=1

issue done: 1
issue done: 2


In [66]:
print(len(issues))
issues[0:2]

2


[{'type': 'Improvement',
  'assignee': 'James Strachan',
  'createdAt': '18/Apr/07 16:06',
  'created_at_epoch': 1176912360,
  'description': None,
  'comments': [{'author': 'Hiram R. Chirino',
    'created_at': '12/Jun/07 18:58',
    'created_at_epoch': 1181674680,
    'text': 'james you think we still need this?'},
   {'author': 'James Strachan',
    'created_at': '01/Oct/07 15:27',
    'created_at_epoch': 1191252420,
    'text': 'seehttp://activemq.apache.org/camel/maven/camel-core/apidocs/org/apache/camel/RuntimeExchangeException.html'}],
  'id': 1,
  'url': 'https://issues.apache.org/jira/browse/CAMEL-1'},
 {'type': 'Bug',
  'assignee': 'Hiram R. Chirino',
  'createdAt': '18/May/07 12:57',
  'created_at_epoch': 1179493020,
  'description': 'Will attach a dump',
  'comments': [],
  'id': 2,
  'url': 'https://issues.apache.org/jira/browse/CAMEL-2'}]

In [67]:
fieldnames = list(issues[0].keys()) 

file_name = f'camel_issues-[{ISSUE_NO_START}-{ISSUE_NO_END}]'
if SPECIFIC_ISSUE_NO:
    file_name = f'camel_issue-{SPECIFIC_ISSUE_NO}'

In [68]:
with open(f'{file_name}.json', 'w', encoding='utf-8') as f:
    json.dump(issues, f, indent=2, ensure_ascii=False)
print(f"{file_name}.json created successfully")

camel_issues-[1-2].json created successfully


In [69]:
with open(f'{file_name}.csv', 'w', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for issue in issues:
        row = issue.copy() 
        row['comments'] = json.dumps(row['comments'])
        writer.writerow(row)
print(f"{file_name}.csv created successfully")

camel_issues-[1-2].csv created successfully
